In [ ]:
# !pip3 install folium
# !pip3 install seaborn

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import folium
from folium import plugins

In [ ]:
df0 = pd.read_csv('./misc_data/202007-citibike-tripdata.csv')

In [ ]:
df1 = df0[df0['starttime'].str.contains('2020-07-01')]

In [ ]:
df = df1

In [ ]:
len(df)

In [ ]:
df['starttime'] = df['starttime'].apply(lambda x: x[:-1].replace(' ','T'))

In [ ]:
df['stoptime'] = df['stoptime'].apply(lambda x: x[:-1].replace(' ','T'))

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
df['start station latitude']

In [ ]:
sns.scatterplot(x = df['start station latitude'], y = df['start station longitude'])

In [ ]:
sns.scatterplot(x = df['end station latitude'], y = df['end station longitude'])

In [ ]:
df.groupby('start station name').count()

In [ ]:
df.groupby('end station name')['tripduration'].count().sort_values(ascending = False)

In [ ]:
df.groupby('start station name')['tripduration'].count().sort_values(ascending = False)

In [ ]:
df.groupby('start station name')['tripduration'].count().sort_values(ascending = False).plot()

In [ ]:
df.groupby('end station name')['tripduration'].count().sort_values(ascending = False).plot()

In [ ]:
df.groupby(['start station name', 'end station name'])['tripduration'].count().sort_values(ascending= False)

In [ ]:
temp = df[['starttime', 'stoptime', 'start station latitude','start station longitude','end station latitude','end station longitude' ]]

In [ ]:
# geo_s = []
# geo_e = []
# for i in range(len(temp)):
#     if i%100 == 0: print(i)
#     geo_s.append([temp.loc[i]['start station latitude'], temp.loc[i]['start station longitude']])
#     geo_e.append([temp.loc[i]['end station latitude'], temp.loc[i]['end station longitude']])

### Data Visualization

In [ ]:
# temp['starttime'] = temp['starttime'].dt.strftime('%Y-%m-%d %H:%M:%S')

In [ ]:
# temp['stoptime'] = temp['stoptime'].dt.strftime('%Y-%m-%d %H:%M:%S')

In [ ]:
temp = temp[temp['stoptime'].str.contains('2020-07-01')]

In [ ]:
temp

In [ ]:
def create_geojson_features_dots_start(df):
    features = []
    for _, row in df.iterrows():
        start_point = [row['start station longitude'],row['start station latitude']]
        feature = {
            'type': 'Feature',
            'geometry': {
                'type': 'Point', 
                'coordinates': start_point
            },
            'properties': {
                'time': row['starttime'],
                'style': {'color' : '#3D98CD'},
                'icon': 'circle',
                'iconstyle':{
                    'fillColor': '#8bc773',
                    'fillOpacity': 0,
                    'stroke': 'true',
                    'radius': 5
                    }
            }
        }
        features.append(feature)
    return features

In [ ]:
def create_geojson_features_dots_end(df):
    features = []
    
    for _, row in df.iterrows():
        end_point = [row['end station longitude'],row['end station latitude']]
        feature = {
            'type': 'Feature',
            'geometry': {
                'type': 'Point', 
                'coordinates': end_point
            },
            'properties': {
                'time': row['stoptime'],
                'style': {'color' : '#c78473'},
                'icon': 'circle',
                'iconstyle':{
                    'fillColor': '#c78473',
                    'fillOpacity': 0,
                    'stroke': 'true',
                    'radius': 5
                    }
            }
        }
        features.append(feature)
    return features

In [ ]:
def create_geojson_features_lines(df):
    features = []
    for _, row in df.iterrows():
        start_point = [row['start station longitude'],row['start station latitude']]
        end_point = [row['end station longitude'],row['end station latitude']]
        feature = {
            'type': 'Feature',
            'geometry': {
                'type':'LineString', #'Point', 
                'coordinates': [start_point, end_point]
            },
            'properties': {
                'times': [row['starttime'], row['starttime']],
                'style': {'color' : '#c78773'}
            }
        }
        features.append(feature)
    return features

In [ ]:
start_geojson = create_geojson_features_dots_start(temp) 
start_geojson[0]
end_geojson = create_geojson_features_dots_end(temp) 
end_geojson[0]

In [ ]:
m = folium.Map(
    location=[40.7504566,-73.9202403],
    zoom_start=12,
    zoom_control=False,
    tiles = "cartodbdark_matter"
)


plugins.TimestampedGeoJson(start_geojson+end_geojson
                           ,period = 'PT10S'
                           ,duration = 'PT1M'
                           ,add_last_point=False
                           ,time_slider_drag_update=True
                           ,transition_time = 10).add_to(m)

m

In [ ]:
# m = folium.Map(
#     location=[40.7504566,-73.9202403],
#     zoom_start=12,
#     zoom_control=False,
#     tiles = "CartoDB Positron"
# )

In [ ]:
m = folium.Map(
    location=[40.7504566,-73.9202403],
    zoom_start=12,
    zoom_control=False,
    tiles = "CartoDB Positron"
)

lines= []
for i in range(len(temp)):
    if i%300 == 0: print(i)
    p1 = [temp.loc[i]['start station latitude'], temp.loc[i]['start station longitude']]
    p2 = [temp.loc[i]['end station latitude'], temp.loc[i]['end station longitude']]
    hmp = {}
    hmp['coordinates'] = [p1, p2]
    hmp['dates'] = [temp.loc[i]['starttime'],temp.loc[i]['stoptime']]
    hmp['color'] = 'red'
    lines.append(hmp)

features = [
    {
        'type': 'Feature',
        'geometry': {
            'type': 'Point', #'LineString',
            'coordinates': line['coordinates'][0],
        },
        'properties': {
            'times': line['dates'],
            'style': {
                'color': line['color'],
                'weight': 5 #line['weight'] if 'weight' in line else 5
            }
        }
    }
    for line in lines
]

# print(features)

plugins.TimestampedGeoJson(
 features
, period='PT1M'
, add_last_point=True).add_to(m)

# # m.save(os.path.join('results', 'Plugins_6.html'))




m

In [ ]:
###################################################

In [ ]:
for p in geo_s:
    folium.Circle(
        radius=5,
        location=[p[0], p[1]],
        color='#73aac9',
        fill=False,
    ).add_to(m)

In [ ]:
for p in geo_e:
    folium.Circle(
        radius=5,
        location=[p[0], p[1]],
        color='#cf907a',
        fill=False,
    ).add_to(m)

In [ ]:
m

In [ ]:
for i in range(len(temp)):
    if i%300 == 0: print(i)
    p1 = [temp.loc[i]['start station latitude'], temp.loc[i]['start station longitude']]
    p2 = [temp.loc[i]['end station latitude'], temp.loc[i]['end station longitude']]
    
    folium.PolyLine([p1,p2], color="#7ab0cf", weight=2.5, opacity=0.2).add_to(m)

In [ ]:
# for p in geo_s:
#     folium.Marker(
#         location=[p[0],p[1]]
#     ).add_to(m)


In [ ]:
# m